# Task 1

## Original exercise number

Exercise 15-3

## Description

1. Write a definition for a type named `Circle` with fields `center` and `radius`, where `center` is a `Point` object and `radius` is a number.

2. Instantiate a circle object that represents a circle with its center at (150,100) and radius 75.

3. Write a function named `pointincircle` that takes a `Circle` object and a `Point` object and returns `true` if the point lies in or on the boundary of the circle.

4. Write a function named `rectincircle` that takes a `Circle` object and a `Rectangle` object and returns `true` if the rectangle lies entirely in or on the boundary of the circle.

5. Write a function named `rectcircleoverlap` that takes a `Circle` object and a `Rectangle` object and returns `true` if any of the corners of the rectangle fall inside the circle. Or as a more challenging version, return `true` if any part of the rectangle falls inside the circle.

## Solution

NO GUARANTEE THAT THE SOLUTION WILL WORK OR WORKS CORRECTLY! USE IT AT
YOUR OWN RISK!

### Imports

In [ ]:
# https://docs.julialang.org/en/v1/stdlib/Test/#Basic-Unit-Tests
import Test as ts 

### Functions

In [ ]:
struct Point{T<:Real}
    x::T
    y::T
end

In [ ]:
function getDistance(p1::Point{<:Real}, p2::Point{<:Real})::Real
    dx::Real = p1.x - p2.x
    dy::Real = p1.y - p2.y
    return sqrt((dx^2) + (dy^2))
end

In [ ]:
struct Rectangle{T<:Real}
    width::T
    height::T
    lowerLeftCorner::Point{<:Real}
end

In [ ]:
struct Circle{T<:Real}
    center::Point{<:Real}
    radius::T
end

In [ ]:
function isPointInCircle(p::Point{<:Real}, c::Circle{<:Real})::Bool
    distFromCenter::Real = getDistance(c.center, p)
    return distFromCenter <= c.radius
end

In [ ]:
# returns vertices, llc, urc, lrc, urc
function get4vertices(rect::Rectangle{<:Real})::Vector{Point{<:Real}}
    llc, w, h = (rect.lowerLeftCorner, rect.width, rect.height)
    vertices::Vector{Point{<:Real}} = [llc]
    for (w1, h1) in zip([w, w, 0], [h, 0, h])
        push!(vertices, Point(llc.x + w1, llc.y + h1))
    end
    return vertices
end

In [ ]:
function isRectInCirc(rect::Rectangle{<:Real}, circ::Circle{<:Real})::Bool
    for v in get4vertices(rect)
        if !isPointInCircle(v, circ)
            return false
        end
    end
    return true
end

## Testing

In [ ]:
circle = Circle(Point(150, 100), 75)
point1 = Point(100, 100)
point2 = Point(300, 300)

In [ ]:
ts.@test isPointInCircle(point1, circle)

In [ ]:
ts.@test !isPointInCircle(point2, circle)

In [ ]:
rectangle1 = Rectangle(75, 50, Point(100, 75))
rectangle2 = Rectangle(75, 50, Point(50, 100))

In [ ]:
ts.@test isRectInCirc(rectangle1, circle)

In [ ]:
ts.@test !isRectInCirc(rectangle2, circle)